In [2]:
# RNN sweep 
# 2021-02-05

# Settings 

# Import master csv file
input_dir = '~/Downloads/'
csv_name = 'New_RNN_sweep_noon.csv'
output_dir = '~/Downloads/'

import pandas as pd
import numpy as np
import os

# Basic data cleaning
rundata = pd.read_csv(os.path.join(input_dir,csv_name))
rundata = rundata[rundata['State']=='finished']


print(len(rundata[rundata['bert_induced']==True]),len(rundata[rundata['bert_induced']==False]))


rundata.columns

5094 744


Index(['Name', 'State', 'Notes', 'User', 'Tags', 'Created', 'Runtime', 'Sweep',
       'batch_size', 'bert_freeze', 'bert_induced', 'bert_model', 'cls_freeze',
       'concat', 'debug', 'device_number', 'dropout', 'embedding_dim',
       'hidden_dim', 'input_path', 'item', 'lr', 'max_length', 'n_epochs',
       'only_BCE', 'path', 'rnn_bidirection', 'rnn_model_type', 'seed',
       'source_file', 'target', 'time_window', 'transformer',
       'transformer_attn_heads', 'transformer_hidden_dim',
       'transformer_layers', 'wandb_project_name', 'eicu_eval_auprc',
       'eicu_eval_auroc', 'eicu_eval_loss', 'eicu_test_auprc',
       'eicu_test_auroc', 'eicu_test_loss', 'eval_auprc', 'eval_auroc',
       'eval_loss', 'mimic_eval_auprc', 'mimic_eval_auroc', 'mimic_eval_loss',
       'mimic_test_auprc', 'mimic_test_auroc', 'mimic_test_loss', 'test_auprc',
       'test_auroc', 'test_loss', 'train_auprc', 'train_auroc', 'train_loss'],
      dtype='object')

In [12]:
def generate_table(result_file:pd.DataFrame, target, seed_count=False, count_flag=True, std=False, rounding=True):
    
    # Conditions: {5 bert models} x {SingleRNN, CLS_finetune} x {4 items} x {5 targets}

    
    # Possible combinations for our models
    conditionlist = [(result_file['bert_induced'] == False),
                     (result_file['bert_induced'] == True) & (result_file['bert_freeze'] == True) & 
                             (result_file['cls_freeze'] == True),
                     (result_file['bert_induced'] == True) & (result_file['bert_freeze'] == True) & 
                             (result_file['cls_freeze'] == False),
                     (result_file['bert_induced'] == True) & (result_file['bert_freeze'] == False)]
    choicelist = ['singleRNN', 'CLSfixed', 'CLSfinetune','BERTfinetune']
    result_file['model_name'] = np.select(conditionlist, choicelist)
    
    
    
    result_file = result_file.loc[:, ['model_name','bert_model','bert_induced','seed', 'source_file', 'target','item',
                                      'eicu_test_auprc', 'mimic_test_auprc', 'test_auprc']]
    
    #print(result_file.loc['bert_model', result_file['model_name']=='singleRNN'].unique())
    result_filter = result_file.loc[result_file['target']==target]
     
    result_mean = pd.pivot_table(result_filter,
                               columns = ['source_file', 'model_name'],
                               index = ['item', 'bert_model'],
                               values = ['test_auprc', 'mimic_test_auprc', 'eicu_test_auprc'],
                               aggfunc=['mean'])
    
    result_std = pd.pivot_table(result_filter,
                               columns = ['source_file', 'model_name'],
                               index = ['item', 'bert_model'],
                               values = ['test_auprc','mimic_test_auprc', 'eicu_test_auprc'],
                               aggfunc=['std'])
    
    result_count = pd.pivot_table(result_filter,
                               columns = ['source_file', 'model_name'],
                               index = ['item', 'bert_model'],
                               values = ['test_auprc','mimic_test_auprc', 'eicu_test_auprc'],
                               aggfunc=['count'])
    
    if count_flag == True:
        if rounding==True:
            RM_drop = result_mean.droplevel([0], axis=1).round(3)
            RS_drop = result_std.droplevel([0], axis=1).round(3)
        elif rounding == False:
            RM_drop = result_mean.droplevel([0], axis=1)
            RS_drop = result_std.droplevel([0], axis=1)
        RC_drop = result_count.droplevel([0], axis=1)
        
        RC_drop = RC_drop[RM_drop.columns]
        
        for col in range(len(RM_drop.columns)): 
            RM_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
            RM_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'   
            
            RS_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
            RS_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'  
    
    
    if seed_count==False:
        if count_flag == True:
            if std==True:
                return RS_drop
            elif std==False:
                return RM_drop
        elif count_flag == False:
            if std==True:
                return result_std
            elif std==False:
                return result_mean
    
    elif seed_count==True:
        return RC_drop


In [24]:
target_list = ['readmission', 'mortality', 'los>3day', 'los>7day', 'dx_depth1_unique']

keep_seed = False
count_seeds = False
flag = True

for target in target_list:
    if keep_seed:
        generate_table(rundata, target).to_csv(os.path.join(output_dir,'RNNsweep_{}_withseed.csv'.format(target)))
    else:
        if count_seeds==False:
            if flag==True:
                generate_table(rundata, target).to_csv(os.path.join(output_dir,'RNNsweep_{}_flagged.csv'.format(target)))
            elif flag==False:
                generate_table(rundata, target).to_csv(os.path.join(output_dir,'RNNsweep_{}.csv'.format(target)))
        if count_seeds==True:
            generate_table(rundata, target).to_csv(os.path.join(output_dir,'RNNsweep_{}_seedcount.csv'.format(target)))
    

<ipython-input-20-1206c978c05a>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RM_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
<ipython-input-20-1206c978c05a>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RM_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'
<ipython-input-20-1206c978c05a>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RS_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
<ipython-input-20-1206c978c

In [13]:
generate_table(rundata, 'mortality').iloc[:, ::-1]

<ipython-input-12-e992721084c9>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RM_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
<ipython-input-12-e992721084c9>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RM_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'
<ipython-input-12-e992721084c9>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RS_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
<ipython-input-12-e99272108

test_auprc                                          \
source_file                    mimic                           eicu            
model_name                 singleRNN CLSfixed CLSfinetune singleRNN CLSfixed   
item    bert_model                                                             
all     bert                     NaN    under       0.439       NaN    0.139   
        bio_bert                 NaN    under       0.436       NaN    0.171   
        bio_clinical_bert        NaN    under       0.443       NaN    0.174   
        blue_bert                NaN    0.396       0.432       NaN    0.181   
        pubmed_bert            0.402    0.433       0.436     0.187    0.167   
inf     bert                     NaN      NaN       0.277       NaN      NaN   
        bio_bert                 NaN      NaN       0.278       NaN      NaN   
        bio_clinical_bert        NaN      NaN       0.278       NaN      NaN   
        blue_bert                NaN      NaN       0.279       NaN      NaN   
        pubmed_bert            0.256      NaN       0.282     0.228      NaN   
lab     bert                     NaN      NaN       0.303       NaN    under   
        bio_bert                 NaN      NaN       0.302       NaN    under   
        bio_clinical_bert        NaN      NaN       0.306       NaN    under   
        blue_bert                NaN    under       0.313       NaN    under   
        pubmed_bert            0.301    under       0.309     0.183    under   
lab_inf bio_bert                 NaN      NaN       0.312       NaN      NaN   
        bio_clinical_bert      0.286      NaN       0.310     0.163      NaN   
        pubmed_bert              NaN      NaN       0.314       NaN      NaN   
med     bert                     NaN      NaN       0.399       NaN      NaN   
        bio_bert                 NaN      NaN       0.389       NaN      NaN   
        bio_clinical_bert        NaN      NaN       0.394       NaN      NaN   
        blue_bert                NaN      NaN       0.394       NaN      NaN   
        pubmed_bert            0.309      NaN       0.394     0.172      NaN   

                                      mimic_test_auprc                       \
source_file                                       both                        
model_name                CLSfinetune        singleRNN CLSfixed CLSfinetune   
item    bert_model                                                            
all     bert                    0.200              NaN    under       0.428   
        bio_bert                0.205              NaN    under        0.43   
        bio_clinical_bert       0.206              NaN    under       0.433   
        blue_bert               0.204              NaN    under       0.423   
        pubmed_bert             0.203            0.390    under        0.43   
inf     bert                    0.265              NaN      NaN       0.279   
        bio_bert                0.264              NaN      NaN       0.277   
        bio_clinical_bert       0.267              NaN      NaN       0.279   
        blue_bert               0.256              NaN      NaN       0.279   
        pubmed_bert             0.266            0.255      NaN        0.28   
lab     bert                    0.222              NaN    under       0.282   
        bio_bert                0.217              NaN    under       0.286   
        bio_clinical_bert       0.226              NaN    under       0.287   
        blue_bert               0.225              NaN    under       0.289   
        pubmed_bert             0.219            0.296    under       0.286   
lab_inf bio_bert                0.231              NaN      NaN       under   
        bio_clinical_bert       0.228            0.274      NaN       under   
        pubmed_bert             0.227              NaN      NaN       under   
med     bert                    0.212              NaN      NaN       0.394   
        bio_bert                0.201           

In [15]:
generate_table(rundata, 'mortality').iloc[:, ::-1].to_latex(multirow=True)

<ipython-input-12-e992721084c9>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RM_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
<ipython-input-12-e992721084c9>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RM_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'
<ipython-input-12-e992721084c9>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RS_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
<ipython-input-12-e99272108

'\\begin{tabular}{llrlrrlrrllrll}\n\\toprule\n    & {} & \\multicolumn{6}{l}{test\\_auprc} & \\multicolumn{3}{l}{mimic\\_test\\_auprc} & \\multicolumn{3}{l}{eicu\\_test\\_auprc} \\\\\n    & source\\_file & \\multicolumn{3}{l}{mimic} & \\multicolumn{3}{l}{eicu} & \\multicolumn{3}{l}{both} & \\multicolumn{3}{l}{both} \\\\\n    & model\\_name &  singleRNN & CLSfixed & CLSfinetune & singleRNN & CLSfixed & CLSfinetune &        singleRNN & CLSfixed & CLSfinetune &       singleRNN & CLSfixed & CLSfinetune \\\\\nitem & bert\\_model &            &          &             &           &          &             &                  &          &             &                 &          &             \\\\\n\\midrule\n\\multirow{5}{*}{all} & bert &        NaN &    under &       0.439 &       NaN &    0.139 &       0.200 &              NaN &    under &       0.428 &             NaN &    under &       0.199 \\\\\n    & bio\\_bert &        NaN &    under &       0.436 &       NaN &    0.171 &       0.205 & 